<img style = "width: 10%" src="cropped-GAS2.png"/>
<b>Grupo de Astrofísica - GAS UFSC</b><br>
Vinicius L. Bilck<br>
v.bilck@grad.ufsc.br<br>

-----------------

## <center> Problema de Dois Corpos - P2 </center>
<div style = 'text-align = justify'><p>Este método de resolução abrange outros referenciais, e leva em consideração ambas massas, pode-se observar a interação das mesmas com relação a massa 1, massa 2, ao raio do centro de massa, e também ao referencial inicial.</p>
Neste caso o método foi utilizado para solucionar a orbita de dois corpos super massivos, no caso, Galáxias </div>

$$\vec{F_{12}} = m_1\cdot \ddot{\vec{r}_1} \\
\vec{F_{21}} = - m_2\cdot \ddot{\vec{r}_2}$$
para a força gravitacional entre as duas massas, temos que
$$\vec{F_g} = - Gm_1m_2\cdot \frac{\vec{r}_1 - \vec{r}_2}{|\vec{r}_1 - \vec{r}_2|^3} \\ \\$$
$$ \\ F_{12} = F_{g} \space ; \space F_{21} = F_{g}$$

Com base nas equações listas acima, há como obter uma EDO para descobrir a $\vec{r_1}(t)$ e $\vec{r_2}(t)$ pois:
$$\ddot{\vec{r_1}} = - m_2\cdot G \frac{\vec{r}_1 - \vec{r}_2}{|\vec{r}_1 - \vec{r}_2|^3} \\$$
$$\\ \ddot{\vec{r_2}} = m_1\cdot G \frac{\vec{r}_1 - \vec{r}_2}{|\vec{r}_1 - \vec{r}_2|^3}$$

In [1]:
from scipy import integrate as i 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from astropy import units, constants
from mpl_toolkits.mplot3d import Axes3D
#plt.style.use('science')
%matplotlib qt

solve_ivp(fun, t_span, y0) $\rightarrow \begin{cases}fun = f \\ t_s = \{t_{i},t_{f}\} \\ y_{0} = [r(t_{i}), \dot{r}(t_{i}), \ddot{r}(t_{i})] \end{cases}$

y0, precisa ser ``` np.shape(y0) = (n,) ```


In [2]:
##EDO
def rtt(t, data, m1, m2, g):
    g_  = g 
    r1 = np.array(data[:3])
    r2 = np.array(data[3:6])
    R = r1 - r2
    r1r2_mod = np.linalg.norm(r1 - r2)
    v1 = np.array(data[6:9])
    v2 = np.array(data[9:12])
    a1 = -(m2 * g_ * R) / r1r2_mod**3
    a2 = (m1 * g_ * R) / r1r2_mod**3

    return np.concatenate([v1, v2, a1, a2])

In [21]:
## Dados
g_pc = 1e10 * constants.G.to((units.kpc**3) / (units.M_sun * units.Gyr**2)).value ## kpc**3 / (Msol * Gyr**2)


##Gal1
m_gal1 = 104 #Msol
x1, y1, z1 = [0, 0, 0]
vx1, vy1, vz1 = [0, 0, 0]

##Gal2       
m_gal2 = 104 #Msol
x2, y2, z2 = [0, -10, 0] #Kpc
vx2, vy2, vz2 = [1200, 0, 0] #Kpc/Gyr


data = [x1, y1, z1, x2, y2, z2, vx1, vy1, vz1, vx2, vy2, vz2]

## EDO Solver
sol = i.solve_ivp(rtt, (0, 1), 
                  data, 
                  args = (m_gal1, m_gal2, g_pc), 
                  t_eval = np.arange(0, 1, 1e-4), method='Radau')   

In [22]:
## Vetores Posição em Relação a (0,0,0)
r_1 = np.column_stack((sol.y[0], sol.y[1], sol.y[2])) #X, Y, Z
v_1 = np.column_stack((sol.y[6], sol.y[7], sol.y[8]))
r_2 = np.column_stack((sol.y[3], sol.y[4], sol.y[5])) #X, Y, Z
v_2 = np.column_stack((sol.y[9], sol.y[10], sol.y[11]))
rcm = ((m_gal1 * r_1) + (m_gal2 * r_2)) / (m_gal1 + m_gal2) # R do centro de massa
vcm = ((m_gal1 * v_1) + (m_gal2 * v_2)) / (m_gal1 + m_gal2) # V do centro de massa

## Em relação a m1
r2r1 = r_2 - r_1
rcmr1 = rcm - r_1

## Em relação a rcm
r1rc = rcm - r_1
r2rc = rcm - r_2
v1rc = vcm - v_1
v2rc = vcm - v_2

print(f"Ultimo ponto em relação ao centro de massa")
print(f"PR1: {r1rc[-1]} | VR1: {-v1rc[-1]}")
print(f"PR2: {r2rc[-1]} | VR2: {-v2rc[-1]}")

Ultimo ponto em relação ao centro de massa
PR1: [-9.10294033  6.7234713   0.        ] | VR1: [ 21.71237637 313.48700861  -0.        ]
PR2: [ 9.10294033 -6.7234713   0.        ] | VR2: [ -21.71237637 -313.48700861   -0.        ]


# Em relação ao Centro de Massa:r2rc

In [23]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot3D(r1rc[:,0],r1rc[:,1], r1rc[:,2], label = "Galaxy 1")
ax.plot3D(r2rc[:,0],r2rc[:,1], r2rc[:,2], label = "Galaxy 2")
ax.scatter(0,0,0, c = 'r')

ax.set_xlabel('x [Kpc]')
ax.set_ylabel('y [Kpc]')
ax.set_zlabel('z [Kpc]')
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.ticklabel_format(axis="z", style="sci", scilimits=(0,0))
plt.legend()
plt.show()

# R1 e R2

In [6]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot3D(r_1[:,0],r_1[:,1], r_1[:,2], label = "Galaxy 1")
ax.plot3D(r_2[:,0],r_2[:,1], r_2[:,2], label = "Galaxy 2")
ax.scatter(0,0,0, c = 'r')

ax.set_xlabel('x [Kpc]')
ax.set_ylabel('y [Kpc]')
ax.set_zlabel('z [Kpc]')
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.ticklabel_format(axis="z", style="sci", scilimits=(0,0))
plt.legend()
plt.show()

In [9]:
numDataPoints = len(sol.y[0])

def animate_func(num):
    ax.clear()
    ax.plot(r1rc[:num,0], r1rc[:num,1], r1rc[:num,2], c='red')
    ax.scatter(r1rc[num-1,0], r1rc[num-1,1], r1rc[num-1,2], c='red')
    ax.plot(r2rc[:num,0], r2rc[:num,1], r2rc[:num,2], c='b')
    ax.scatter(r2rc[num-1,0], r2rc[num-1,1], r2rc[num-1,2], c='b')

    ax.set_xlabel('x [Km]')
    ax.set_ylabel('y [Km]')
    ax.set_zlabel('z [Km]')
    ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    ax.ticklabel_format(axis="z", style="sci", scilimits=(0,0))
    plt.title(f'{num} Dias')

# Plotting the Animation
fig = plt.figure()
ax = plt.axes(projection='3d')
line_ani = animation.FuncAnimation(fig, animate_func, interval=1,   
                                   frames=numDataPoints)
plt.show()